### 유아기 (생후 12개월 ~ 48개월) 모유수유 여부 변수 전처리

#### 1. 국민건강영양조사 - 기본DB (2012 ~ 2017) 의 Y_MTM_YN 칼럼 사용
#### 2. 이후 시군구 코드 outcome_sudogwon_idx 과 merge
#### 3. 모유수유 여부 이외에 유아기 식생활에 관련된 변수까지 같이 고려해서 concat (영양제 섭취 여부 등) 

In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm

In [2]:
기본12 = pd.read_sas('Breastfeeding/hn12_all.sas7bdat', format = 'sas7bdat')
기본13 = pd.read_sas('Breastfeeding/HN13_ALL.sas7bdat', format = 'sas7bdat')
기본14 = pd.read_sas('Breastfeeding/HN14_ALL.sas7bdat', format = 'sas7bdat')
기본15 = pd.read_sas('Breastfeeding/HN15_ALL.sas7bdat', format = 'sas7bdat')
기본16 = pd.read_sas('Breastfeeding/hn16_all.sas7bdat', format = 'sas7bdat')
기본17 = pd.read_sas('Breastfeeding/hn17_all.sas7bdat', format = 'sas7bdat')

In [3]:
outcome_sudogwon_idx = pd.read_csv('outcome_sudogwon_idx.csv', encoding = 'euc-kr')

### 12년도

In [4]:
기본12

,mod_d,ID,ID_fam,year,region,town_t,apt_t,psu,sex,age,...,LF_S10,LF_S11,LF_S12,LF_S13,LF_S14,LF_S14_1,LF_S15,LF_S16,LF_SECUR,LF_SECUR_G
0,b'2019.03.06.',b'A108964301',b'A1089643',2012.0,1.0,1.0,2.0,b'A108',1.0,67.0,...,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,2.0,1.0
1,b'2019.03.06.',b'A108964302',b'A1089643',2012.0,1.0,1.0,2.0,b'A108',2.0,63.0,...,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,2.0,1.0
2,b'2019.03.06.',b'A108964303',b'A1089643',2012.0,1.0,1.0,2.0,b'A108',2.0,35.0,...,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,2.0,1.0
3,b'2019.03.06.',b'A108966301',b'A1089663',2012.0,1.0,1.0,2.0,b'A108',1.0,33.0,...,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,0.0,1.0
4,b'2019.03.06.',b'A108966302',b'A1089663',2012.0,1.0,1.0,2.0,b'A108',2.0,32.0,...,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8053,b'2019.03.06.',b'P111978901',b'P1119789',2012.0,16.0,2.0,1.0,b'P111',1.0,75.0,...,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,0.0,1.0
8054,b'2019.03.06.',b'P111978902',b'P1119789',2012.0,16.0,2.0,1.0,b'P111',2.0,67.0,...,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,0.0,1.0
8055,b'2019.03.06.',b'P111979301',b'P1119793',2012.0,16.0,2.0,1.0,b'P111',1.0,63.0,...,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,0.0,1.0
8056,b'2019.03.06.',b'P111979302',b'P1119793',2012.0,16.0,2.0,1.0,b'P111',2.0,59.0,...,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,0.0,1.0


In [5]:
기본12['Y_MTM_YN'].unique()  # 모유수유 변수 unique 값 체크

array([nan,  1.,  2.])

In [6]:
tmp = pd.DataFrame(기본12[['ID', 'region', 'town_t', 'psu', 'Y_MTM_YN']])  # 필요한 열만 뽑기

In [7]:
# 모유수유 여부가 1, 2인 행만 뽑기 (무응답 제외하기)

tmp2 = tmp.loc[(tmp['Y_MTM_YN'] == 1) | (tmp['Y_MTM_YN'] == 2)].reset_index(drop = True)
tmp2

,ID,region,town_t,psu,Y_MTM_YN
0,b'A108967503',1.0,1.0,b'A108',1.0
1,b'A108972906',1.0,1.0,b'A108',1.0
2,b'A108974103',1.0,1.0,b'A108',2.0
3,b'A108975903',1.0,1.0,b'A108',1.0
4,b'A109967504',1.0,1.0,b'A109',1.0
...,...,...,...,...,...
277,b'P109962903',16.0,1.0,b'P109',2.0
278,b'P111972304',16.0,2.0,b'P111',1.0
279,b'P111972305',16.0,2.0,b'P111',1.0
280,b'P111976905',16.0,2.0,b'P111',1.0


In [8]:
# region 변수를 숫자 -> 문자로 바꾸기

tmp2.loc[tmp2['region'] == 1, 'region'] = '서울시'
tmp2.loc[tmp2['region'] == 2, 'region'] = '부산시'
tmp2.loc[tmp2['region'] == 3, 'region'] = '대구시'
tmp2.loc[tmp2['region'] == 4, 'region'] = '인천시'
tmp2.loc[tmp2['region'] == 5, 'region'] = '광주시'
tmp2.loc[tmp2['region'] == 6, 'region'] = '대전시'
tmp2.loc[tmp2['region'] == 7, 'region'] = '울산시'
tmp2.loc[tmp2['region'] == 8, 'region'] = '세종시'
tmp2.loc[tmp2['region'] == 9, 'region'] = '경기도'
tmp2.loc[tmp2['region'] == 10, 'region'] = '강원도'
tmp2.loc[tmp2['region'] == 11, 'region'] = '충청북도'
tmp2.loc[tmp2['region'] == 12, 'region'] = '충청남도'
tmp2.loc[tmp2['region'] == 13, 'region'] = '전라북도'
tmp2.loc[tmp2['region'] == 14, 'region'] = '전라남도'
tmp2.loc[tmp2['region'] == 15, 'region'] = '경상북도'
tmp2.loc[tmp2['region'] == 16, 'region'] = '경상남도'
tmp2.loc[tmp2['region'] == 17, 'region'] = '제주도'

In [9]:
tmp2

,ID,region,town_t,psu,Y_MTM_YN
0,b'A108967503',서울시,1.0,b'A108',1.0
1,b'A108972906',서울시,1.0,b'A108',1.0
2,b'A108974103',서울시,1.0,b'A108',2.0
3,b'A108975903',서울시,1.0,b'A108',1.0
4,b'A109967504',서울시,1.0,b'A109',1.0
...,...,...,...,...,...
277,b'P109962903',경상남도,1.0,b'P109',2.0
278,b'P111972304',경상남도,2.0,b'P111',1.0
279,b'P111972305',경상남도,2.0,b'P111',1.0
280,b'P111976905',경상남도,2.0,b'P111',1.0


In [17]:
# 지역별, 모유수유 여부별 sum

tmp2.rename(columns = {'Y_MTM_YN' : '모유 수유 여부'}, inplace = True)
pd.DataFrame(tmp2.groupby(['region', '모유 수유 여부']).count()['ID'])

ID
region 모유 수유 여부    
강원도    1.0        4
       2.0        1
경기도    1.0        7
       2.0        1
경상남도   1.0        4
       2.0        1
경상북도   1.0        9
       2.0        2
광주시    1.0        7
대구시    1.0       10
대전시    1.0        8
부산시    1.0       16
서울시    1.0       49
       2.0        6
세종시    1.0       81
       2.0        7
울산시    1.0        4
인천시    1.0       13
       2.0        1
전라남도   1.0       17
       2.0        1
전라북도   1.0        7
충청남도   1.0        7
충청북도   1.0       15
       2.0        4

In [11]:
outcome_sudogwon_idx.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [12]:
outcome_sudogwon_idx['시군구'].unique()

array(['경기도 가평군', '서울특별시 강남구', '서울특별시 강동구', '서울특별시 강북구', '서울특별시 강서구',
       '인천광역시 계양구', '경기도 고양시 덕양구', '경기도 고양시 일산동구', '경기도 과천시', '서울특별시 관악구',
       '경기도 광명시', '경기도 광주시', '서울특별시 광진구', '서울특별시 구로구', '경기도 구리시',
       '경기도 군포시', '서울특별시 금천구', '경기도 김포시', '인천광역시 남동구', '경기도 남양주시',
       '서울특별시 노원구', '서울특별시 도봉구', '인천광역시 동구', '서울특별시 동대문구', '경기도 동두천시',
       '서울특별시 동작구', '서울특별시 마포구', '인천광역시 남구', '경기도 부천시 소사구', '인천광역시 부평구',
       '인천광역시 서구', '서울특별시 서대문구', '서울특별시 서초구', '경기도 성남시 분당구',
       '경기도 성남시 수정구', '경기도 성남시 중원구', '서울특별시 성동구', '서울특별시 성북구',
       '서울특별시 송파구', '경기도 수원시 권선구', '경기도 수원시 영통구', '경기도 수원시 장안구',
       '경기도 수원시 팔달구', '경기도 시흥시', '경기도 안산시 단원구', '경기도 안산시 상록구', '경기도 안성시',
       '경기도 안양시 동안구', '경기도 안양시 만안구', '경기도 양주시', '서울특별시 양천구', '인천광역시 연수구',
       '경기도 연천군', '서울특별시 영등포구', '경기도 오산시', '서울특별시 용산구', '경기도 용인시 기흥구',
       '경기도 용인시 수지구', '경기도 용인시 처인구', '서울특별시 은평구', '경기도 의왕시', '경기도 의정부시',
       '경기도 이천시', '서울특별시 종로구', '서울특별시 중구', '인천광역시 중구', '서울특별시 중랑구',
       '경기도 파주시', '경기도 평택시', 

**국건영 데이터에 구별 구분 칼럼이 없음**    
**일단 서울시, 경기도, 인천시 만이라도 붙여보자**

In [13]:
# outcome_sudogwon_idx 에서 시군구 칼럼 국건영 데이터와 동일하게 바꾸기

outcome_sudogwon_idx.loc[outcome_sudogwon_idx['시군구'].str.startswith('서울특별시'), '시군구'] = '서울시'
outcome_sudogwon_idx.loc[outcome_sudogwon_idx['시군구'].str.startswith('경기도'), '시군구'] = '경기도'
outcome_sudogwon_idx.loc[outcome_sudogwon_idx['시군구'].str.startswith('인천광역시'), '시군구'] = '인천시'

In [14]:
outcome_sudogwon_idx

,air_out_idx,시군구,SIGUNGU_CD,SIGUNGU_NM,sudogwon_check
0,1,경기도,31370,가평군,1
1,2,서울시,11230,강남구,1
2,3,서울시,11250,강동구,1
3,5,서울시,11090,강북구,1
4,6,서울시,11160,강서구,1
...,...,...,...,...,...
67,231,경기도,31200,파주시,1
68,233,경기도,31070,평택시,1
69,234,경기도,31270,포천시,1
70,237,경기도,31180,하남시,1
